In [ ]:
from pylab import *

import skimage
from skimage import color
from skimage import exposure 
from skimage import filters
from skimage import io
from skimage import util
from skimage import restoration

import numpy as np
from scipy import ndimage

import time

In [ ]:
img = io.imread('../sekvence/mm.jpg')

M,N,piksel = np.shape(img) #dimenzije slike

#konverzija slike u HSV kolor sistem
imghsv = color.rgb2hsv(img)

#prikaz ulazne slike
#plt.figure()
#io.imshow(img)
#plt.axis('off')
#plt.title('Ulazna slika')

#prikaz i cuvanje Hue komponente slike
#plt.figure()
#io.imshow(imghsv[:,:,0], vmin = 0, vmax = 1, cmap='jet')
#plt.axis('off')
#plt.title('H komponenta')
#plt.imsave('img_hue.png', imghsv[:,:,0], cmap='jet', vmin=0, vmax=1)

#prikaz i cuvanje Saturation komponente slike
#plt.figure()
#io.imshow(imghsv[:,:,1], vmin = 0, vmax = 1, cmap='jet')
#plt.axis('off')
#plt.title('S komponenta')
#plt.imsave('img_saturation.png', imghsv[:,:,1], cmap='jet', vmin=0, vmax=1)

#prikaz i cuvanje Value komponente slike
#plt.figure()
#io.imshow(imghsv[:,:,2], vmin = 0, vmax = 1, cmap='gray')
#plt.axis('off')
#plt.title('V komponenta')
#plt.imsave('img_value.png', imghsv[:,:,2], cmap='gray', vmin=0, vmax=1)

#glavna obrada slike - za sve piksele posmatramo saturaciju i hue
for i in range(0,M):
    for j in range(0,N):
        if imghsv[i,j,1] > 0.3: #ako je saturacija veca od praga 0.3 (izuzimamo bele delove slike)
            if imghsv[i,j,0] > 0.5 and imghsv[i,j,0] < 0.75: #ako je hue u opsegu [0.5, 0.75] - plava boja na slici
                imghsv[i,j,0] = 0.85 #postavimo hue na 0.85 - ljubicasta boja


#konverzija slike u RGB kolor sistem
img_out = color.hsv2rgb(imghsv)

#prikaz i cuvanje izlazne slike uporedo sa ulaznom slikom
plt.figure()
plt.subplot(1,2,1)
io.imshow(img)
plt.axis('off')
plt.title('Ulazna slika')
plt.subplot(1,2,2)
io.imshow(img_out)
plt.axis('off')
plt.title('Izlazna slika')
#plt.imsave('mm_out.jpg', img_out)

## Zadatak 2 - popravka kvaliteta slike

In [ ]:
def restore(img):

    M,N = np.shape(img) #dimenzije slike
    
    #prikaz i cuvanje medjurezultata
    #plt.figure()
    #io.imshow(img, cmap='gray', vmin=0, vmax=255)
    #plt.axis('off')
    #plt.imsave('img.png', img, cmap='gray', vmin=0, vmax=255)
    
    #median filtar za uklanjanje crno-belog suma
    img_median = filters.median(img, footprint=None, out=None, mode='nearest', cval=0.0, behavior='ndimage')

    #prikaz i cuvanje medjurezultata
    #plt.figure()
    #io.imshow(img_median, cmap='gray', vmin=0, vmax=255)
    #plt.axis('off')
    #plt.imsave('img_median.png', img_median, cmap='gray', vmin=0, vmax=255)
    
    #for petlja za invertovanje vrednosti - crni pikseli postaju beli i obrnuto
    img_invert = img_median
    for i in range(M): 
        for j in range(N):
            img_invert[i,j] = 255-img_median[i,j]
    
    #prikaz i cuvanje medjurezultata
    #plt.figure()
    #io.imshow(img_invert, cmap='gray', vmin=0, vmax=255)
    #plt.axis('off')
    #plt.imsave('img_invert.png', img_invert, cmap='gray', vmin=0, vmax=255)
    
    #parametri za razvlacenje histograma
    low_in = uint8(np.percentile(img_invert.flatten(), 1)) #1% piksela ima manju vrednost od low_in
    high_in = uint8(np.percentile(img_invert.flatten(), 99)) #99% piksela ima manju vrednost od high_in
    #razvlacenje histograma
    img_out = exposure.rescale_intensity(img_invert, in_range = (low_in, high_in), out_range = (0, 255))
    
    #prikaz i cuvanje krajnjeg rezultata
    #plt.figure()
    #plt.imshow(img_out, cmap='gray')
    #plt.axis('off')
    #plt.imsave('img_out.png', img_out, cmap='gray', vmin=0, vmax=255)
    
    #izlazna slika
    return img_out

### Testiranje funkcije restore na primeru

In [ ]:
img = io.imread('../sekvence/corrupted1.png')
img_out = restore(img)

plt.figure()
plt.imshow(img_out, cmap='gray')
_ = plt.axis('off')
#plt.imsave('corrupted3_out.png', img_out, cmap='gray', vmin=0, vmax=255)

## Zadatak 3

In [ ]:
def bilateral_filter(x, radius, sigma_s, sigma_r):
    #proveravamo da li su ulazni parametri validni
    if sigma_s <= 0:
        print("Greska - sigma_s mora biti pozitivno")
        return
    if sigma_r <= 0:
        print("Greska - sigma_r mora biti pozitivno")
        return 
    if type(x) != np.ndarray or x.min() < 0 or x.max() > 1:
        print("Greska - Ulazna slika mora biti tipa float unutar opsega [0,1]")
        return 
    if type(radius) != int or radius < 0:
        print("Greska - radius mora biti pozitivan ceo broj")
        return 
    
    #dimenzije ulazne slike
    M, N = np.shape(x)
    img_out = np.zeros((M,N))
    
    #maska Gausovog filtra
    mask_gaus = np.zeros((2*radius+1, 2*radius+1))
    for k in range(-radius, radius+1):
        for l in range(-radius, radius+1):
            mask_gaus[k+radius, l+radius] = np.exp(-(k**2+l**2)/(2*sigma_s**2))
    
    #proširivanje slike za vrednost radijusa sa svake strane
    padded_x = np.pad(x, pad_width=radius, mode='reflect')
    
    #for petlja za iteraciju krpz sliku
    for m in range(M):
        for n in range(N):
            #koordinate unutar proširene slike
            i = m + radius
            j = n + radius
            #maska za uracunavanje razlike u intenzitetima
            mask_intensity = np.exp(-((padded_x[i-radius:i+radius+1, j-radius:j+radius+1] - padded_x[i, j])**2) / (2 * sigma_r**2))
            #konacna maska
            mask_bilateral = mask_gaus * mask_intensity
            #normalizacija parametara - suma maske je 1
            mask_bilateral /= mask_bilateral.sum()
            #vrednost u koordinati m,n je korelacija maske i dela slike oko m,n
            img_out[m, n] = np.sum(mask_bilateral * padded_x[i-radius:i+radius+1, j-radius:j+radius+1])
    
    return img_out

### Testiranje bilateralnog filtra na primeru

In [ ]:
img = io.imread('../sekvence/lena.tif')
img = img/256

#prikaz ulazne slike
plt.figure()
plt.imshow(img, cmap='gray', vmin=0, vmax = 1)
_ = plt.axis('off')

#prikaz i cuvanje rada filtera sa parametrima: r = 3, sigma_s = 1, sigma_r = 0.4
img_1 = bilateral_filter(img, 3, 1, 0.4)
plt.figure()
plt.imshow(img_1, cmap='gray', vmin=0, vmax=1)
_ = plt.axis('off')
#plt.imsave('img_1.png', img_1, cmap='gray', vmin=0, vmax=1)

#prikaz i cuvanje rada filtera sa parametrima: r = 3, sigma_s = 3, sigma_r = 0.4
img_2 = bilateral_filter(img, 3, 3, 0.4)
plt.figure()
plt.imshow(img_2, cmap='gray', vmin=0, vmax=1)
_ = plt.axis('off')
#plt.imsave('img_2.png', img_2, cmap='gray', vmin=0, vmax=1)

#prikaz i cuvanje rada filtera sa parametrima: r = 3, sigma_s = 3, sigma_r = 0.1
img_3 = bilateral_filter(img, 3, 3, 0.1)
plt.figure()
plt.imshow(img_3, cmap='gray', vmin=0, vmax=1)
_ = plt.axis('off')
#plt.imsave('img_3.png', img_3, cmap='gray', vmin=0, vmax=1)

#prikaz i cuvanje rada filtera sa parametrima: r = 10, sigma_s = 1, sigma_r = 5
img_4 = bilateral_filter(img, 3,1,5)
plt.figure()
plt.imshow(img_4, cmap='gray', vmin=0, vmax=1)
_ = plt.axis('off')
#plt.imsave('img_4.png', img_4, cmap='gray', vmin=0, vmax=1)

#prikaz i cuvanje rada filtera sa parametrima: r = 10, sigma_s = 1, sigma_r = 0.4
img_5 = bilateral_filter(img, 10, 2, 0.4)
plt.figure()
plt.imshow(img_5, cmap='gray', vmin=0, vmax=1)
_ = plt.axis('off')
#plt.imsave('img_5.png', img_5, cmap='gray', vmin=0, vmax=1)

### Testiranje brzine rada funkcije bilateral_filter u zavisnosti od radijusa

In [ ]:
x_data = np.arange(1,41)
y_data = np.zeros(40,dtype=float)
y_built_in = np.zeros(40,dtype=float)
#formiranje grafika iteracijom po radiusu
for i in range(1,40):
    radius = i
    #belezimo vreme izvrsavanja rucne funkcije
    start = time.time() 
    x = bilateral_filter(img,radius,2*radius,0.1)
    end = time.time()
    execution_time = end - start
    y_data[i] = execution_time
    
    #belezimo vreme izvrsavanja ugradjene funkcije
    start = time.time()
    x = restoration.denoise_bilateral(img,radius,0.1,2*radius)
    end = time.time()
    execution_time = end - start
    y_built_in[i] = execution_time

In [ ]:
#prikaz podataka
plt.figure()
plt.plot(x_data, y_data, y_built_in)
plt.legend(['Filtar iz zadatka', 'Ugradjena funkcija filtra'])
plt.xlabel('Radius')
plt.ylabel('Vreme [s]')
grid()
plt.show()